### Imports

In [ ]:
import numpy as np
from specmf.models import Graph, MultiFidelityModel
from specmf.utils import error_analysis, load_model_config
from specmf.data import load_data
from specmf.plot import *

In [ ]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 14px;
</style>

#### Data loading

In [ ]:
dataset_name = "cavity-flow"

x_LF, x_HF = load_data(
    dataset_name,
    preprocess=True,
    normalize=False, 
    flatten=True,
)

print(f"{x_LF.shape=}", f"{x_HF.shape=}")

### Plot data 

In [ ]:
plot_data(x_LF, x_HF, dataset_name, n_samples=3)

### Multi-fidelity model

#### Create graph and model instances

In [ ]:
# Create the graph
graph_config = {
    'metric': 'euclidean',
    'dist_space': 'ambient',
    'n_components': None,
    'method': 'full',
    'k_nn': None,
    'corr_scale': None,
    'k_adj': 7,
    'p': 0.5,
    'q': 0.5,
}

g_LF = Graph(data=x_LF, **graph_config)

#### Perform spectral clustering

In [ ]:
n_HF = 100
inds_train, labels = g_LF.cluster(n_HF)

In [ ]:
plot_cluster_size_hist(labels)

In [ ]:
# Plot Laplacian spectrum
eigvals, eigvecs = g_LF.laplacian_eig()
plot_spectrum(eigvals, 50)

#### Split high-fidelity data

In [ ]:
n_samples = x_HF.shape[0]
inds_test = np.delete(np.arange(n_samples), inds_train)
x_HF_train = x_HF[inds_train, :]
x_HF_test = x_HF[inds_test, :]

print(f"{x_HF_train.shape=}", f"{x_HF_test.shape=}")

#### Fit and train the model

In [ ]:
fit_model = False

# Create the model 
model_config = {
    'sigma': 0.01,
    'method': 'full'
}
model = MultiFidelityModel(**model_config)

In [ ]:
if fit_model:
    x_MF, C_phi, dPhi, loss_history, kappa_history = model.fit_transform(
        g_LF,
        x_HF_train,
        inds_train,
        maxiter=10,
        step_size=1e3,
        step_decay_rate=1.0,
        ftol=1e-6,
        gtol=1e-8,
        verbose=False,
    )
    model.summary()

    plot_loss_and_kappa(loss_history, kappa_history)

else:
    from pathlib import Path

    notebook_path = Path.cwd()
    yaml_file_path = notebook_path.parent.parent / 'data/model_configs.yaml'
    model_config = load_model_config(yaml_file_path, dataset_name)
    
    model = MultiFidelityModel(**model_config)

    x_MF, C_phi, dPhi = model.transform(g_LF, x_HF_train, inds_train)
    model.summary()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hist(dPhi, bins=20)
ax.set_xlabel("Variance", fontsize=14)
ax.set_ylabel("Frequency", fontsize=14)
ax.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
ax.tick_params(axis="both", labelsize=12)
ax.set_title("Variance histogram Cavity data", fontsize=18)

### Results

In [ ]:
# Error Analysis for unseen test datadet
error_analysis(x_LF[inds_test], x_MF[inds_test], x_HF_test)

In [ ]:
# Error Analysis for the whole dataset
error_analysis(x_LF, x_MF, x_HF)

In [ ]:
E_LF = 100 * np.linalg.norm(x_LF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)
E_MF = 100 * np.linalg.norm(x_MF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)

plot_distributions(E_LF, E_MF, bins_LF=25, bins_MF=25, mask=None)

In [ ]:
import matplotlib.pyplot as plt


plt.rcParams.update({
    "figure.facecolor": (0, 0, 0, 0),   # Transparent
    "axes.facecolor":   (0, 0, 0, 0),   # Transparent
    "savefig.facecolor": (0, 0, 0, 0),  # Transparent when saving
    "axes.labelcolor":  "white",
    "axes.edgecolor":   "white",
    "axes.titlecolor":  "white",
    "xtick.color":      "white",
    "ytick.color":      "white",
    # "ztick.color":      "white",
})

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"

lf_color = "navajowhite"
hf_color = "red"
mf_color = "lime"

def _plot_mf_comparison_curves(
    X_LF: np.ndarray,
    X_MF: np.ndarray,
    X_HF: np.ndarray,
    x_values: np.ndarray,
    xlabel: str,
    ylabel: str,
    samples: list[int] | None = None,
    ymax: float | None = None,
    ymin: float | None = -1.2,
    legend_loc: str = "lower right",
    figsize: tuple[float, float] = (12.5, 12),
    grid: bool = True,
) -> None:
    """
    General function to plot multi-fidelity comparison curves.

    Parameters:
    - X_LF: Low-fidelity data (2D array).
    - X_MF: Multi-fidelity data (2D array).
    - X_HF: High-fidelity data (2D array).
    - x_values: Array of x-axis values for plotting.
    - xlabel: Label for the x-axis.
    - ylabel: Label for the y-axis.
    - samples: List of samples to plot (default: None).
    - ymax: Maximum value for the y-axis (default: None).
    - ymin: Minimum value for the y-axis (default: None).
    - legend_loc: Location for the legend in the last subplot (default: 'lower right').
    - figsize: Size of the figure (default: (12.5, 8)).
    - grid: Whether to show grid (default: True).
    """

    fig, axs = plt.subplots(3, 2, figsize=figsize)
    fig.subplots_adjust(wspace=0.075, hspace=0.075)

    ymin_ = np.inf if ymin is None else ymin
    ymax_ = -np.inf if ymax is None else ymax

    # Loop over the 2x2 grid of subplots
    for i, (r, c) in enumerate(np.ndindex(3, 2)):
        j = samples[i]

        # Plot low, multi, and high-fidelity curves
        axs[r, c].plot(x_values, X_LF[j], label="Low-Fidelity", c='navajowhite')
        axs[r, c].plot(x_values, X_MF[j], label="Multi-Fidelity", c='lime')
        axs[r, c].plot(x_values, X_HF[j], label="High-Fidelity", c='cyan')

        axs[r, c].grid(grid)

        # Configure axis labels
        if r == 2:
            axs[r, c].set_xlabel(xlabel, fontsize=22)
        if c == 0:
            axs[r, c].set_ylabel(ylabel, rotation=0, fontsize=22, labelpad=20)

        # Hide tick labels for the appropriate axes
        if r < 2:
            axs[r, c].set_xticklabels([])
        if c > 0:
            axs[r, c].set_yticklabels([])

        # Calculate the minimum value for consistent y-axis limits
        if ymin is None:
            ymin_ = min(ymin_, 1.2 * np.min([X_LF[j], X_HF[j]]))
        if ymax is None:
            ymax_ = max(ymax_, 1.2 * np.max([X_LF[j], X_HF[j]]))

    # Set consistent y-limits for all subplots
    for ax in axs.flat:
        ax.set_ylim([ymin_, ymax_])

    # Add legend in the bottom-right subplot
    axs[2, 1].legend(
        loc=legend_loc,
        fontsize=18,
        facecolor="none",
        edgecolor="white",
        labelcolor="white",
    )

    plt.show()

# Comparison plotting functions
def plot_mf_comparison(
    X_LF: np.ndarray,
    X_MF: np.ndarray,
    X_HF: np.ndarray,
    dataset_name: str,
    **kwargs,
) -> None:
    """Plot multi-fidelity comparisons based on the dataset name."""

    _plot_mf_comparison_curves(X_LF, X_MF, X_HF, **kwargs)
    plt.show()

In [ ]:
x_values = np.linspace(0, 1, x_LF.shape[1])
ylabel = r"$\varphi_{q}(y)$"
xlabel = r"$y$"
ymax = 0.
samples = [6371, 233, 2374, 2272, 5791, 8073]  # [6371, 1726, 5791, 8073]

plot_mf_comparison(
    dataset_name=dataset_name,
    X_LF=x_LF,
    X_MF=x_MF,
    X_HF=x_HF,
    samples=samples,
    x_values=x_values,
    xlabel=xlabel,
    ylabel=ylabel,
    ymax=ymax,
)


In [ ]:
n_points, _ = x_LF.shape
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
fig.subplots_adjust(wspace=0.15)
vmin, vmax = 1 * np.min((x_LF, x_HF)), 1.2 * np.max((x_LF, x_HF))
for i in range(100):
    j = np.random.randint(0, n_points)
    x_ = np.linspace(0, 1, x_LF[j, :].shape[0])
    axs[0].plot(x_, x_LF[j, :], label="Low-Fidelity")
    axs[0].set_title("Low-Fidelity")
    axs[1].plot(x_, x_HF[j, :], label="High-Fidelity")
    axs[1].set_title("High-Fidelity")
    axs[2].plot(x_, x_MF[j, :], label="Multi-Fidelity")
    axs[2].set_title("Multi-Fidelity")
    axs[0].set_ylim((vmin, vmax))
    axs[1].set_ylim((vmin, vmax))
    axs[2].set_ylim((vmin, vmax))

### Visualize uncertainty of multi-fidelity estimates

In [ ]:
import matplotlib.ticker as ticker

def fmt(x, pos):
    a, b = '{:.2e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [ ]:
_, eigevs = g_LF.laplacian_eig()
X = np.real(eigvecs[:, :n_HF])

In [ ]:
import umap

X_umap = umap.UMAP(
    n_components=2,
    n_neighbors=50,
    min_dist=1,
    # metric="precomputed",
    init='random', 
    random_state=42
).fit_transform(X)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 9.5))
ax1 = ax.scatter(X_umap[:, 0], X_umap[:, 1], s=15, c=dPhi, vmin=0.02, vmax=0.037)
ax.scatter(X_umap[inds_train, 0], X_umap[inds_train, 1], s=30, c='r', marker='o')
cb = plt.colorbar(ax1, format=ticker.FuncFormatter(fmt))
cb.ax.tick_params(labelsize=16)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('UMAP of Cavity Flow data (Case 5)', fontsize=26, pad=10)